In [ ]:
import random

In [1]:
SEARCH_SPACE = {
    'k_size_a': [1, 3, 5, 7],
    'k_size_b': [1, 3, 5, 7],
    'out_channels_a': [8, 16, 32, 64],
    'out_channels_b': [8, 16, 32, 64],
    'include_pool_a': [True, False],
    'include_pool_b': [True, False],
    'pool_type_a': ['max_pooling','avg_pooling'],
    'pool_type_b': ['max_pooling','avg_pooling'],
    'activation_type_a': ['relu', 'tanh', 'elu', 'selu'],
    'activation_type_b': ['relu', 'tanh', 'elu', 'selu'],
    'include_b': [True, False],
    'include_BN_a': [True, False],
    'include_BN_b': [True, False],
    'skip_connection': [True, False],
}

In [ ]:
class Generation():
  def __init__(self,
               fit_survival_rate: float,
               unfit_survival_rate: float,
               mutation_rate: float,
               pop_size: int,
               phase: int,
               search_space: dict,
               prev_best: Chromosome):
    self.fit_survival_rate = fit_survival_rate
    self.unfit_survival_rate = unfit_survival_rate
    self.mutation_rate = mutation_rate
    self.pop_size = pop_size
    self.pop = []

    for i in range(pop_size):
      self.pop.append(Chromosome(phase = 0,
                            prev_best = prev_best,
                            genes = self.make_gene(search_space)))
      
  def make_gene(search_space: dict):
    gene = {}
    
    gene['k_size_a'] = random.choice(search_space['k_size_a'])
    gene['k_size_b'] = random.choice(search_space['k_size_b'])
    gene['out_channels_a'] = random.choice(search_space['out_channels_a'])
    gene['out_channels_b'] = random.choice(search_space['out_channels_b'])
    gene['include_pool_a'] = random.choice(search_space['include_pool_a'])
    gene['include_pool_b'] = random.choice(search_space['include_pool_b'])
    gene['pool_type_a'] = random.choice(search_space['pool_type_a'])
    gene['pool_type_b'] = random.choice(search_space['pool_type_b'])
    gene['activation_type_a'] = random.choice(search_space['activation_type_a'])
    gene['activation_type_b'] = random.choice(search_space['activation_type_b'])
    gene['include_b'] = random.choice(search_space['include_b'])
    gene['include_bn_a'] = random.choice(search_space['include_bn_a'])
    gene['include_bn_b'] = random.choice(search_space['include_bn_b'])
    gene['skip_connection'] = random.choice(search_space['skip_connection'])

    return gene

  def sort_pop(self, train_loader, test_loader):
    sorted_pop = sorted(self.pop,
                        key = lambda x: x.fitness_function(train_loader, test_loader))
    self.pop = sorted_pop

  def generate(self):
    num_fit_selected = int(self.fit_survival_rate * self.pop_size)
    num_unfit_selected = int(self.unfit_survival_rate * self.pop_size)
    num_mutate = int(self.mutation_rate * self.pop_size)

    new_pop = []

    for i in range(num_fit_selected):
      new_pop.append(self.pop[i])

    for i in range(num_unfit_selected):
      new_pop.append(self.pop[self.pop_size - i - 1])

    if(num_mutate > len(new_pop)):
      indices_to_mutate = random.sample(range(0, len(new_pop)), num_mutate)
    else:
      indices_to_mutate = random.sample(range(0, len(new_pop)), len(new_pop))

    for i in indices_to_mutate:
      new_pop[i].mutate()

    parents_list = []
    for i in range(self.pop_size):
      parents = random.sample(range(0, len(new_pop)), 2)
      parents_list.append(tuple(parents))

    for p1, p2 in parents_list:
      new_pop.append(new_pop[p1].crossover(new_pop[p2]))


SyntaxError: ignored